# Goal

The goal is to take in a series of inputs for each player available for purchase in FPL -> and turn that into a prediction for their points for the gameweek. 

# What is needed?

In order to generate an expected point value for a player, we need data about players and what they scored each week. <br>

It does not seem like this sort of information is being saved anywhere. As such, the first phase of this project will be setting up the pipeline to collect this data each gameweek. We will want to collect a bunch of information from a few different sources, things like percentage of minutes played, xG Per 90, xA per 90, "threat", "influence, "creativity (those 3 being FPL generated metrics), opposition xG conceded, home or away, etc. <br>

We will want to be able to collect this weekly as a snapshot BEFORE the matches are played. After they are played, we will append a "points_scored" to each record. Eventually we aim to be able to predict this points scored value give all the data we collect, but we need the data in the week-by-week format in order to do this. 

# Phase 1: Week-by-week Historical Data Collection

## 1) Data sources and desired attributes

Here I will outline the specific data sources I am going to pull from, and what data I want. 

### Fbref

Think of this site as providing data from two perspectives: team and individual. <br>

As for team data, we want to have attributes that give an idea of how the individual's team is performing, but also how the team they are playing against is performing. Therefore..

- all expected stats per 90 minutes FOR (don't even pull goals and assists, I just care about expected). We will use this to see how good of an attacking team this player is playing for, and how bad of an attacking team they are playing against
- all expected stats per 90 minutes AGAINST (tells us how good or bad of a defense this player plays for or is up against)

And for the individual perspective:

- percentage of minutes played this season - "min%" (is the player playing a lot?)
- expected stats per 90 (how effective is this player attacking-wise?)
- tackle + challenge + blocks, per 90 data (how effective is this player defensively?)
- yellow/ red cards per 90 (these actions lose points, so we want to know about them)
- penalty share, a number between 0 and 1 (we want to know if a player is their team's penalty kick taker, as this is a good way to get points)

We will also get all the scheduling information out of this site. 

### Official fantasy premier league site

We want to know some stuff as it relates to the game itself. These include:

- price and selection %, won't really assist in predicting points (or rather we don't want to use them for that) but will come in handy for later functionality with the model, like picking differentials and building a squad
- FORM - very important. We want to know how this player is performing coming into the gameweek
- finally, actual points scored.

Remember, these are all snapshot statistics - we want to know what these values were before the gameweek, and after the gameweek, we want to append the points scored to each record. 

### Proposed workflow

1) A script runs to start to fill out the games to be played in the next gameweek. It fills in a record for each player, with the gameweek, individual's team, and opposition.

2) We then access the Fbref data source in order to get team and opposition data. Basically, we will match on the player's team first, getting expected data both for and against - then we repeat the process for the opposition.

3) Now, we have the player, who they are playing, and data about how their team is performing per 90 and how their opposition is performing per 90 up to this point in the season. We should now attach all the data from the player perspective to each row. Get all the per 90 data. This should all be quite simple except for the penalty kick share, which will require a simple calculation to see what perfecntage of a team's penalty kicks the player has taken.

4) Now, join in the data from the official FPL website. Match based on player name, and grab price, % selection, form, and the column "points_scored" but leave this BLANK (we will not know it at the time this script runs).

5) We will let the game week happen, then run the script that gets player points for the week from the official FPL site. Join this in based on player name to the records we just created, using matchweek and player name as the combined key. 

In [1]:
import soccerdata as sd
import pandas as pd
from datetime import datetime
import requests
from thefuzz import process
import warnings

curr_gameweek = 12

warnings.simplefilter(action='ignore', category=FutureWarning)

def get_fixtures(week_wanted):
    """
    grabs the list of games for the week, extracts only the cleaned team names of home and away team, as well as match_week, 
    """
    fbref = sd.FBref(leagues='ENG-Premier League', seasons='2025-2026')
    schedule = fbref.read_schedule()
    schedule['date'] = pd.to_datetime(schedule['date'], errors='coerce')
    schedule = schedule[schedule['week'] == week_wanted]

    return schedule[['home_team','away_team','week']]


def get_fbref_player_stats(season='2025-2026',pt_threshold=40):
    """
    grabs all player individual statistics that we want
    """
    fbref = sd.FBref('ENG-Premier League', season)

    standard = fbref.read_player_season_stats(stat_type="standard")
    shooting = fbref.read_player_season_stats(stat_type="shooting")
    passing = fbref.read_player_season_stats(stat_type="passing")
    defense = fbref.read_player_season_stats(stat_type="defense")
    playing_time = fbref.read_player_season_stats(stat_type="playing_time")

    def flatten_cols(df):
        df = df.copy()
        df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns.values]
        return df

    standard = flatten_cols(standard)
    shooting = flatten_cols(shooting)
    passing = flatten_cols(passing)
    defense = flatten_cols(defense)
    playing_time = flatten_cols(playing_time)

    for df in [standard, shooting, passing, defense, playing_time]:
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'player'}, inplace=True)

    metadata_cols = ['season', 'league', 'team', 'nation_', 'pos_', 'age_', 'born_']
    for df in [standard, shooting, passing, defense]:
        df.drop(columns=[c for c in metadata_cols if c in df.columns], inplace=True)

    fbref_stats = standard
    for df in [shooting, passing, defense, playing_time]:
        fbref_stats = fbref_stats.merge(df, on='player', how='outer')

    
    fbref_stats['Tackles_Tkl_per90'] = fbref_stats['Tackles_Tkl'] / fbref_stats['Playing Time_90s_y']
    fbref_stats['Blocks_Blocks_per90'] = fbref_stats['Blocks_Blocks'] / fbref_stats['Playing Time_90s_y']
    fbref_stats['yellow_per90'] = fbref_stats['Performance_CrdY'] / fbref_stats['Playing Time_90s_y']
    fbref_stats['red_per90'] = fbref_stats['Performance_CrdR'] / fbref_stats['Playing Time_90s_y']

    fbref_stats = fbref_stats[fbref_stats['Playing Time_Min%'] >= pt_threshold]

    name_map = {
        "Alisson": "Alisson Becker",
        "André": "André Trindade da Costa Neto",
        "Benjamin Šeško": "Benjamin Sesko",
        "Bernardo Silva": "Bernardo Mota Veiga de Carvalho e Silva",
        "Beto": "Norberto Bercique Gomes Betuncal",
        "Bruno Guimarães": "Bruno Guimarães Rodriguez Moura",
        "Casemiro": "Carlos Henrique Casimiro",
        "David Raya": "David Raya Martín",
        "Diego Gómez": "Diego Gómez Amarilla",
        "Diogo Dalot": "Diogo Dalot Teixeira",
        "Emi Buendía": "Emiliano Buendía Stati",
        "Evanilson": "Francisco Evanilson de Lima Barbosa",
        "Ezri Konsa": "Ezri Konsa Ngoyo",
        "Ferdi Kadioglu": "Ferdi Kadıoğlu",
        "Florentino Luís": "Florentino Ibrain Morris Luís",
        "Gabriel Magalhães": "Gabriel dos Santos Magalhães",
        "Hugo Bueno": "Hugo Bueno López",
        "Jeremy Doku": "Jérémy Doku",
        "Joelinton": "Joelinton Cássio Apolinário de Lira",
        "Joshua King": "Josh King",
        "João Gomes": "Gustavo Nunes Fernandes Gomes",
        "João Palhinha": "João Maria Lobo Alves Palhares Costa Palhinha Gonçalves",
        "João Pedro": "João Pedro Junqueira de Jesus",
        "Lucas Paquetá": "Lucas Tolentino Coelho de Lima", 
        "Lucas Perri": "Lucas Estella Perri",
        "Marc Cucurella": "Marc Cucurella Saseta",
        "Mateus Fernandes": "Mateus Gonçalo Espanha Fernandes",
        "Matheus Cunha": "Matheus Santos Carneiro da Cunha",
        "Max Kilman": "Maximilian Kilman",
        "Moisés Caicedo": "Moisés Caicedo Corozo",
        "Morato": "Felipe Rodrigues Da Silva",
        "Murillo": "Murillo Costa dos Santos",
        "Nicolás González": "Nico González Iglesias",
        "Pedro Neto": "Pedro Lomba Neto",
        "Pedro Porro": "Pedro Porro Sauceda",
        "Raúl Jiménez": "Raúl Jiménez Rodríguez",
        "Richarlison": "Richarlison de Andrade",
        "Rúben Dias": "Rúben dos Santos Gato Alves Dias",
        "Santiago Bueno": "Santiago Ignacio Bueno",
        "Thiago": "Igor Thiago Nascimento Rodrigues",
        "Valentino Livramento": "Tino Livramento",
        "Yeremi Pino": "Yéremy Pino Santos",
        "Álex Jiménez": "Álex Jiménez Sánchez"
    }

    fbref_stats['player'] = fbref_stats['player'].apply(lambda x: name_map.get(x, x))

    return fbref_stats[['player','Playing Time_Min%','Per 90 Minutes_xG','Per 90 Minutes_xAG','Tackles_Tkl_per90','Blocks_Blocks_per90','yellow_per90','red_per90']]
    
def get_teams(season="2025-2026"):
    fbref = sd.FBref("ENG-Premier League", season)
    
    teams = fbref.read_team_season_stats(stat_type="shooting").index.tolist()
    
    data = []
    today = pd.Timestamp(datetime.today().date())
    
    for team in teams:
        matches = fbref.read_team_match_stats(stat_type="schedule", team=team)
        
        matches = matches[pd.to_datetime(matches["date"]) <= today]
        
        if matches.empty:
            continue
        
        xG_for_per90 = matches["xG"].sum() / len(matches)
        xG_against_per90 = matches["xGA"].sum() / len(matches)
        
        data.append({
            "team": team,
            "xG_for_per90": xG_for_per90,
            "xG_against_per90": xG_against_per90
        })

    df = pd.DataFrame(data)
    
    df['team'] = df['team'].astype(str).str.replace(r'^.*,\s*(.*?)\)$', r'\1', regex=True)

    df['team'] = df['team'].str.replace("'", "").str.strip()

    name_map = {
    "Manchester Utd": "Man Utd",
    "Manchester City": "Man City",
    "Tottenham": "Spurs",
    "Nott'ham Forest": "Nott'm Forest",
    "Newcastle Utd": "Newcastle",
    "Leeds United" : "Leeds"}
    
    df['team_name'] = df['team'].replace(name_map)

    return df


def get_players():
    """
    Grabs a list of all FPL players
    """
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    response = requests.get(url)
    data = response.json()
    
    players = pd.DataFrame(data['elements'])
    teams = {team['id']: team['name'] for team in data['teams']}
    players['team_name'] = players['team'].map(teams)
    
    positions = {pos['id']: pos['singular_name'] for pos in data['element_types']}
    players['position'] = players['element_type'].map(positions)
    
    players_df = players[['id', 'first_name', 'second_name', 'team_name', 'position', 'now_cost']].copy()
    players_df['full_name'] = players_df['first_name'] + " " + players_df['second_name']
    
    return players_df


def fuzzy_match(fpl_df, fbref_df, threshold=92):
    """
    Fuzzy matches FPL players (already mapped) to FBref player stats by name
    """
    
    fbref_names = fbref_df['player'].tolist()
    
    fpl_names = fpl_df['full_name'].tolist()

    mapping = {}
    for name in fpl_names:
        if pd.isna(name):
            mapping[name] = None
            continue
        match, score = process.extractOne(name, fbref_names)
        mapping[name] = match if score >= threshold else None

    fpl_df['matched_fbref'] = fpl_df['full_name'].map(mapping)

    merged = fpl_df.merge(fbref_df, left_on='matched_fbref', right_on='player', how='inner')

    merged = merged.drop_duplicates(subset=['player'])

    return merged

import pandas as pd
import requests

def get_fpl_table():
    """Pulls current PL standings from the official FPL API."""
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    data = requests.get(url).json()

    teams = pd.DataFrame(data["teams"])[["name", "short_name", "position"]]

    # Normalize names to match your naming scheme
    name_map = {
        "Manchester United": "Man Utd",
        "Manchester City": "Man City",
        "Tottenham Hotspur": "Spurs",
        "Nottingham Forest": "Nott'm Forest",
        "Newcastle United": "Newcastle",
        "Leeds United": "Leeds",
        "Brighton and Hove Albion": "Brighton",
        "Wolverhampton Wanderers": "Wolves",
        "West Ham United": "West Ham",
        "Aston Villa": "Aston Villa",
        "Sheffield United": "Sheffield Utd",
    }

    teams["team_name"] = teams["name"].replace(name_map)

    return teams[["team_name", "position"]]


def get_fixtures_and_league_spots(gameweek=curr_gameweek):
    fixtures = get_fixtures(gameweek)

    name_map = {
        "Manchester Utd": "Man Utd",
        "Manchester City": "Man City",
        "Tottenham": "Spurs",
        "Nott'ham Forest": "Nott'm Forest",
        "Newcastle Utd": "Newcastle",
        "Leeds United": "Leeds"
    }

    fixtures['home_team'] = fixtures['home_team'].replace(name_map)
    fixtures['away_team'] = fixtures['away_team'].replace(name_map)

    home_df = fixtures[["home_team", "week"]].rename(columns={"home_team": "team"})
    home_df["home"] = 1

    away_df = fixtures[["away_team", "week"]].rename(columns={"away_team": "team"})
    away_df["home"] = 0

    combined = pd.concat([home_df, away_df], ignore_index=True)

    standings = get_fpl_table()

    final = combined.merge(standings, left_on="team", right_on="team_name", how="left")
    final = final.drop(columns=["team_name"])

    return final.sort_values("position")


def join_it_all_together():
    df_fpl = get_players()
    df_fbref = get_fbref_player_stats()
    df_teams = get_teams()
    df_fuz = fuzzy_match(df_fpl,df_fbref)
    df = df_fuz.merge(df_teams,left_on="team_name",right_on="team",how="left")

    df_fix = get_fixtures_and_league_spots() 

    df = df.merge(df_fix,on='team',how='left')

    df = df.rename(columns={'team':'team_name','position_x':'player_position','now_cost':'current_fpl_cost',
                        'Playing Time_Min%':'playing_time_min_percentage','Per 90 Minutes_xG':'xg_per_90',
                       'Per 90 Minutes_xAG':'xag_per_90','Tackles_Tkl_per90':'tackles_per_90','Blocks_Blocks_per90':'blocks_per_90',
                       'yellow_per90':'yellows_per_90','red_per90':'reds_per_90','xG_for_per90':'team_xg_per_90',
                       'xG_against_per90':'team_xg_against_per_90','week':'gameweek','home':'is_at_home',
                       'position_y':'team_league_position'})
    
    return df[['full_name','team_name','player_position','current_fpl_cost','playing_time_min_percentage','xg_per_90','xag_per_90','blocks_per_90',
          'yellows_per_90','reds_per_90','team_xg_per_90','team_xg_against_per_90','gameweek','is_at_home','team_league_position']]


[01/21/26 18:38:36] INFO     No custom team name replacements found. You can configure these in       ]8;id=284117;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=987073;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_config.py#91\91]8;;\
                             /home/tars/soccerdata/config/teamname_replacements.json.                              

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=595086;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=913878;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_config.py#197\197]8;;\
                             /home/tars/soccerdata/config/league_dict.json.                                        

In [2]:
df = join_it_all_together()
df.shape

[01/21/26 18:38:37] INFO     Saving cached data to /home/tars/soccerdata/data/FBref                  ]8;id=638554;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=163653;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py#263\263]8;;\

[01/21/26 18:38:38] INFO     Saving cached data to /home/tars/soccerdata/data/FBref                  ]8;id=186683;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=218684;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py#263\263]8;;\

[01/21/26 18:38:45] ERROR    Error while scraping                                                    ]8;id=297578;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=779162;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py#545\545]8;;\
                             https://fbref.com/en/squads/18bb7c10/2025-2026/matchlogs/all_comps/sche               
                             dule. Retrying... (attempt 1 of 5).                                                   
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "/home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py"               
                             , line 525, in _download_and_save                                                     
                                 response.raise_for_status()                                                       
                               File                                                                                
                             "/home/tars/jupyter/lib/python3.12/site-packages/requests/models.py",                 
                             line 1024, in raise_for_status                                                        
                                 raise HTTPError(http_error_msg, response=self)                                    
                             requests.exceptions.HTTPError: 403 Client Error: Forbidden for url:                   
                             https://fbref.com/en/squads/18bb7c10/2025-2026/matchlogs/all_comps/sche               
                             dule                                                                                  

[01/21/26 18:38:51] ERROR    Error while scraping                                                    ]8;id=152476;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=952189;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py#545\545]8;;\
                             https://fbref.com/en/squads/18bb7c10/2025-2026/matchlogs/all_comps/sche               
                             dule. Retrying... (attempt 2 of 5).                                                   
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "/home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py"               
                             , line 525, in _download_and_save                                                     
                                 response.raise_for_status()                                                       
                               File                                                                                
                             "/home/tars/jupyter/lib/python3.12/site-packages/requests/models.py",                 
                             line 1024, in raise_for_status                                                        
                                 raise HTTPError(http_error_msg, response=self)                                    
                             requests.exceptions.HTTPError: 403 Client Error: Forbidden for url:                   
                             https://fbref.com/en/squads/18bb7c10/2025-2026/matchlogs/all_comps/sche               
                             dule                                                                                  

[01/21/26 18:38:57] ERROR    Error while scraping                                                    ]8;id=157698;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=556153;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py#545\545]8;;\
                             https://fbref.com/en/squads/18bb7c10/2025-2026/matchlogs/all_comps/sche               
                             dule. Retrying... (attempt 3 of 5).                                                   
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "/home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py"               
                             , line 525, in _download_and_save                                                     
                                 response.raise_for_status()                                                       
                               File                                                                                
                             "/home/tars/jupyter/lib/python3.12/site-packages/requests/models.py",                 
                             line 1024, in raise_for_status                                                        
                                 raise HTTPError(http_error_msg, response=self)                                    
                             requests.exceptions.HTTPError: 403 Client Error: Forbidden for url:                   
                             https://fbref.com/en/squads/18bb7c10/2025-2026/matchlogs/all_comps/sche               
                             dule                                                                                  

[01/21/26 18:39:04] ERROR    Error while scraping                                                    ]8;id=896896;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=502523;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py#545\545]8;;\
                             https://fbref.com/en/squads/18bb7c10/2025-2026/matchlogs/all_comps/sche               
                             dule. Retrying... (attempt 4 of 5).                                                   
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "/home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py"               
                             , line 525, in _download_and_save                                                     
                                 response.raise_for_status()                                                       
                               File                                                                                
                             "/home/tars/jupyter/lib/python3.12/site-packages/requests/models.py",                 
                             line 1024, in raise_for_status                                                        
                                 raise HTTPError(http_error_msg, response=self)                                    
                             requests.exceptions.HTTPError: 403 Client Error: Forbidden for url:                   
                             https://fbref.com/en/squads/18bb7c10/2025-2026/matchlogs/all_comps/sche               
                             dule                                                                                  

[01/21/26 18:39:10] ERROR    Error while scraping                                                    ]8;id=835399;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=395163;file:///home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py#545\545]8;;\
                             https://fbref.com/en/squads/18bb7c10/2025-2026/matchlogs/all_comps/sche               
                             dule. Retrying... (attempt 5 of 5).                                                   
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "/home/tars/jupyter/lib/python3.12/site-packages/soccerdata/_common.py"               
                             , line 525, in _download_and_save                                                     
                                 response.raise_for_status()                                                       
                               File                                                                                
                             "/home/tars/jupyter/lib/python3.12/site-packages/requests/models.py",                 
                             line 1024, in raise_for_status                                                        
                                 raise HTTPError(http_error_msg, response=self)                                    
                             requests.exceptions.HTTPError: 403 Client Error: Forbidden for url:                   
                             https://fbref.com/en/squads/18bb7c10/2025-2026/matchlogs/all_comps/sche               
                             dule                                                                                  

ConnectionError: Could not download https://fbref.com/en/squads/18bb7c10/2025-2026/matchlogs/all_comps/schedule.

In [ ]:
# all that is left todo.. 

# 1) A single method that is meant to be ran after the gameweek (the above is an before) that grabs the points scored for each player and associates
#    them together

# 2) Save this a SQlite db or Postgres db.. something running on the server. Save the .db file in the github too.. for extra safety. 

In [15]:
def get_players_with_points(gameweek):
    """
    Returns FPL players + their total points for a specific gameweek.
    """
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    response = requests.get(url)
    data = response.json()

    # Base players df
    players = pd.DataFrame(data['elements'])
    teams = {team['id']: team['name'] for team in data['teams']}
    players['team_name'] = players['team'].map(teams)

    positions = {pos['id']: pos['singular_name'] for pos in data['element_types']}
    players['position'] = players['element_type'].map(positions)

    # Build base DF
    players_df = players[['id', 'first_name', 'second_name', 'team_name', 'position', 'now_cost']].copy()
    players_df['full_name'] = players_df['first_name'] + " " + players_df['second_name']

    # Get points for each player in the given gameweek
    points_list = []
    for pid in players_df['id']:
        p_url = f"https://fantasy.premierleague.com/api/element-summary/{pid}/"
        p_data = requests.get(p_url).json()

        history = p_data.get("history", [])
        gw_record = next((gw for gw in history if gw["round"] == gameweek), None)

        points_list.append(gw_record["total_points"] if gw_record else 0)

    players_df["gw_points"] = points_list
    
    return players_df[['full_name','gw_points']]

df_ = get_players_with_points(11)

In [17]:
df_.head(30)

,full_name,gw_points
0,David Raya Martín,1
1,Kepa Arrizabalaga Revuelta,0
2,Karl Hein,0
3,Tommy Setford,0
4,Gabriel dos Santos Magalhães,1
5,William Saliba,1
6,Riccardo Calafiori,1
7,Jurriën Timber,1
8,Jakub Kiwior,0
9,Myles Lewis-Skelly,0


In [20]:
X = pd.read_csv("/home/tars/Projects/fpl-oracle/X_12.csv")
filtered = df_[df_['full_name'].isin(X['full_name'])]
filtered.shape

(246, 2)

# Phase 1.5: Fbref betrayal

In [ ]:
# fbref removed advanced statistics. We need to pull them from elsewhere now. 

import logging

logging.getLogger("soccerdata").setLevel(logging.WARNING)

import requests
import pandas as pd
from pathlib import Path
import soccerdata as sd
import pandas as pd
from datetime import datetime
import requests
from thefuzz import process
import warnings
import re

def get_gameweeks_seen(data_dir="/data"):
    """
    Returns a sorted list of gameweek numbers found as X_<gw>.csv in data_dir
    """
    pattern = re.compile(r"X_(\d+)\.csv$")
    gameweeks = []

    for path in Path(data_dir).iterdir():
        match = pattern.match(path.name)
        if match:
            gameweeks.append(int(match.group(1)))

    return sorted(gameweeks)

def get_next_gameweek():
    """
    Returns the next FPL gameweek number based on current date.
    """
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    data = requests.get(url).json()

    events = pd.DataFrame(data["events"])

    next_gw = events.loc[events["is_next"], "id"]

    if next_gw.empty:
        raise ValueError("No upcoming gameweek found")

    return int(next_gw.iloc[0]) 

curr_gameweek = get_next_gameweek()

def get_fixtures(week_wanted):
    """
    grabs the list of games for the week, extracts only the cleaned team names of home and away team, as well as match_week, 
    """
    fbref = sd.FBref(leagues='ENG-Premier League', seasons='2025-2026')
    schedule = fbref.read_schedule()
    schedule['date'] = pd.to_datetime(schedule['date'], errors='coerce')
    schedule = schedule[schedule['week'] == week_wanted]

    return schedule[['home_team','away_team','week']]

"""
def get_fbref_player_stats(season='2025-2026',pt_threshold=60):
    """
    grabs all player individual statistics that we want
    """
    fbref = sd.FBref('ENG-Premier League', season)

    standard = fbref.read_player_season_stats(stat_type="standard")
    shooting = fbref.read_player_season_stats(stat_type="shooting")
    passing = fbref.read_player_season_stats(stat_type="passing")
    defense = fbref.read_player_season_stats(stat_type="defense")
    playing_time = fbref.read_player_season_stats(stat_type="playing_time")

    def flatten_cols(df):
        df = df.copy()
        df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns.values]
        return df

    standard = flatten_cols(standard)
    shooting = flatten_cols(shooting)
    passing = flatten_cols(passing)
    defense = flatten_cols(defense)
    playing_time = flatten_cols(playing_time)

    for df in [standard, shooting, passing, defense, playing_time]:
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'player'}, inplace=True)

    metadata_cols = ['season', 'league', 'team', 'nation_', 'pos_', 'age_', 'born_']
    for df in [standard, shooting, passing, defense]:
        df.drop(columns=[c for c in metadata_cols if c in df.columns], inplace=True)

    fbref_stats = standard
    for df in [shooting, passing, defense, playing_time]:
        fbref_stats = fbref_stats.merge(df, on='player', how='outer')

    
    fbref_stats['Tackles_Tkl_per90'] = fbref_stats['Tackles_Tkl'] / fbref_stats['Playing Time_90s_y']
    fbref_stats['Blocks_Blocks_per90'] = fbref_stats['Blocks_Blocks'] / fbref_stats['Playing Time_90s_y']
    fbref_stats['yellow_per90'] = fbref_stats['Performance_CrdY'] / fbref_stats['Playing Time_90s_y']
    fbref_stats['red_per90'] = fbref_stats['Performance_CrdR'] / fbref_stats['Playing Time_90s_y']

    fbref_stats = fbref_stats[fbref_stats['Playing Time_Min%'] >= pt_threshold]

    name_map = {
        "Alisson": "Alisson Becker",
        "André": "André Trindade da Costa Neto",
        "Benjamin Šeško": "Benjamin Sesko",
        "Bernardo Silva": "Bernardo Mota Veiga de Carvalho e Silva",
        "Beto": "Norberto Bercique Gomes Betuncal",
        "Bruno Guimarães": "Bruno Guimarães Rodriguez Moura",
        "Casemiro": "Carlos Henrique Casimiro",
        "David Raya": "David Raya Martín",
        "Diego Gómez": "Diego Gómez Amarilla",
        "Diogo Dalot": "Diogo Dalot Teixeira",
        "Emi Buendía": "Emiliano Buendía Stati",
        "Evanilson": "Francisco Evanilson de Lima Barbosa",
        "Ezri Konsa": "Ezri Konsa Ngoyo",
        "Ferdi Kadioglu": "Ferdi Kadıoğlu",
        "Florentino Luís": "Florentino Ibrain Morris Luís",
        "Gabriel Magalhães": "Gabriel dos Santos Magalhães",
        "Hugo Bueno": "Hugo Bueno López",
        "Jeremy Doku": "Jérémy Doku",
        "Joelinton": "Joelinton Cássio Apolinário de Lira",
        "Joshua King": "Josh King",
        "João Gomes": "Gustavo Nunes Fernandes Gomes",
        "João Palhinha": "João Maria Lobo Alves Palhares Costa Palhinha Gonçalves",
        "João Pedro": "João Pedro Junqueira de Jesus",
        "Lucas Paquetá": "Lucas Tolentino Coelho de Lima", 
        "Lucas Perri": "Lucas Estella Perri",
        "Marc Cucurella": "Marc Cucurella Saseta",
        "Mateus Fernandes": "Mateus Gonçalo Espanha Fernandes",
        "Matheus Cunha": "Matheus Santos Carneiro da Cunha",
        "Max Kilman": "Maximilian Kilman",
        "Moisés Caicedo": "Moisés Caicedo Corozo",
        "Morato": "Felipe Rodrigues Da Silva",
        "Murillo": "Murillo Costa dos Santos",
        "Nicolás González": "Nico González Iglesias",
        "Pedro Neto": "Pedro Lomba Neto",
        "Pedro Porro": "Pedro Porro Sauceda",
        "Raúl Jiménez": "Raúl Jiménez Rodríguez",
        "Richarlison": "Richarlison de Andrade",
        "Rúben Dias": "Rúben dos Santos Gato Alves Dias",
        "Santiago Bueno": "Santiago Ignacio Bueno",
        "Thiago": "Igor Thiago Nascimento Rodrigues",
        "Valentino Livramento": "Tino Livramento",
        "Yeremi Pino": "Yéremy Pino Santos",
        "Álex Jiménez": "Álex Jiménez Sánchez"
    }

    fbref_stats['player'] = fbref_stats['player'].apply(lambda x: name_map.get(x, x))

    return fbref_stats[['player','Playing Time_Min%','Per 90 Minutes_xG','Per 90 Minutes_xAG','Tackles_Tkl_per90','Blocks_Blocks_per90','yellow_per90','red_per90']]
"""

"""
def get_teams(season="2025-2026"):
    fbref = sd.FBref("ENG-Premier League", season)
    
    teams = fbref.read_team_season_stats(stat_type="shooting").index.tolist()
    
    data = []
    today = pd.Timestamp(datetime.today().date())
    
    for team in teams:
        matches = fbref.read_team_match_stats(stat_type="schedule", team=team)
        
        matches = matches[pd.to_datetime(matches["date"]) <= today]
        
        if matches.empty:
            continue
        
        xG_for_per90 = matches["xG"].sum() / len(matches)
        xG_against_per90 = matches["xGA"].sum() / len(matches)
        
        data.append({
            "team": team,
            "xG_for_per90": xG_for_per90,
            "xG_against_per90": xG_against_per90
        })

    df = pd.DataFrame(data)
    
    df['team'] = df['team'].astype(str).str.replace(r'^.*,\s*(.*?)\)$', r'\1', regex=True)

    df['team'] = df['team'].str.replace("'", "").str.strip()

    name_map = {
    "Manchester Utd": "Man Utd",
    "Manchester City": "Man City",
    "Tottenham": "Spurs",
    "Nott'ham Forest": "Nott'm Forest",
    "Newcastle Utd": "Newcastle",
    "Leeds United" : "Leeds"}
    
    df['team_name'] = df['team'].replace(name_map)

    return df
"""

def get_players():
    """
    Grabs a list of all FPL players
    """
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    response = requests.get(url)
    data = response.json()
    
    players = pd.DataFrame(data['elements'])
    teams = {team['id']: team['name'] for team in data['teams']}
    players['team_name'] = players['team'].map(teams)
    
    positions = {pos['id']: pos['singular_name'] for pos in data['element_types']}
    players['position'] = players['element_type'].map(positions)
    
    players_df = players[['id', 'first_name', 'second_name', 'team_name', 'position', 'now_cost']].copy()
    players_df['full_name'] = players_df['first_name'] + " " + players_df['second_name']
    
    return players_df


def fuzzy_match(fpl_df, fbref_df, threshold=92):
    """
    Fuzzy matches FPL players (already mapped) to FBref player stats by name
    """
    
    fbref_names = fbref_df['player'].tolist()
    
    fpl_names = fpl_df['full_name'].tolist()

    mapping = {}
    for name in fpl_names:
        if pd.isna(name):
            mapping[name] = None
            continue
        match, score = process.extractOne(name, fbref_names)
        mapping[name] = match if score >= threshold else None

    fpl_df['matched_fbref'] = fpl_df['full_name'].map(mapping)

    merged = fpl_df.merge(fbref_df, left_on='matched_fbref', right_on='player', how='inner')

    merged = merged.drop_duplicates(subset=['player'])

    return merged

def get_fpl_table():
    """Pulls current PL standings from the official FPL API."""
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    data = requests.get(url).json()

    teams = pd.DataFrame(data["teams"])[["name", "short_name", "position"]]

    name_map = {
        "Manchester United": "Man Utd",
        "Manchester City": "Man City",
        "Tottenham Hotspur": "Spurs",
        "Nott'ham Forest": "\"Nottham Forest\"",
        "Nottingham Forest": "\"Nottham Forest\"",
        "Nott'm Forest": "\"Nottham Forest\"",
        "Newcastle United": "Newcastle",
        "Leeds United": "Leeds",
        "Brighton and Hove Albion": "Brighton",
        "Wolverhampton Wanderers": "Wolves",
        "West Ham United": "West Ham",
        "Aston Villa": "Aston Villa",
        "Sheffield United": "Sheffield Utd",
    }

    teams["team_name"] = teams["name"].replace(name_map)

    return teams[["team_name", "position"]]


def get_fixtures_and_league_spots(gameweek=curr_gameweek):
    fixtures = get_fixtures(gameweek)
    
    name_map = {
        "Manchester Utd": "Man Utd",
        "Manchester City": "Man City",
        "Tottenham": "Spurs",
        "Nott'ham Forest": "\"Nottham Forest\"",
        "Nottingham Forest": "\"Nottham Forest\"",
        "Newcastle Utd": "Newcastle",
        "Leeds United": "Leeds"
    }

    fixtures['home_team'] = fixtures['home_team'].replace(name_map)
    fixtures['away_team'] = fixtures['away_team'].replace(name_map)

    home_df = fixtures[["home_team", "week"]].rename(columns={"home_team": "team"})
    home_df["home"] = 1

    away_df = fixtures[["away_team", "week"]].rename(columns={"away_team": "team"})
    away_df["home"] = 0

    combined = pd.concat([home_df, away_df], ignore_index=True)

    standings = get_fpl_table()

    final = combined.merge(standings, left_on="team", right_on="team_name", how="left")
    final = final.drop(columns=["team_name"])

    return final.sort_values("position")


def join_it_all_together():
    TEAM_TEST_MAP = {
    "Manchester Utd": "Man Utd",
    "Manchester City": "Man City",
    "Tottenham": "Spurs",
    "Nott'm Forest": "\"Nottham Forest\"",
    "Newcastle Utd": "Newcastle",
    "Leeds United": "Leeds",

    "Manchester United": "Man Utd",
    "Manchester City": "Man City",
    "Tottenham Hotspur": "Spurs",
    "Nottingham Forest": "\"Nottham Forest\"",
    "Newcastle United": "Newcastle",
    "Leeds United": "Leeds",
}

    df_fpl = get_players()
    df_fbref = get_fbref_player_stats()
    df_teams = get_teams()
    df_fuz = fuzzy_match(df_fpl,df_fbref)

    """
    print("Team names seen in df_fuz")
    print(df_fuz['team_name'].value_counts())
    print()

    print("Team names seen in df_teams")
    print(df_teams['team'].value_counts())
    print()
    """

    df_fuz["team_name"] = df_fuz["team_name"].replace(TEAM_TEST_MAP)
    df_teams["team"] = df_teams["team"].replace(TEAM_TEST_MAP)
    
    df = df_fuz.merge(df_teams,left_on="team_name",right_on="team",how="left")

    df_fix = get_fixtures_and_league_spots() 

    df = df.merge(df_fix,on='team',how='left')

    df = df.rename(columns={'team':'team_name','position_x':'player_position','now_cost':'current_fpl_cost',
                        'Playing Time_Min%':'playing_time_min_percentage','Per 90 Minutes_xG':'xg_per_90',
                       'Per 90 Minutes_xAG':'xag_per_90','Tackles_Tkl_per90':'tackles_per_90','Blocks_Blocks_per90':'blocks_per_90',
                       'yellow_per90':'yellows_per_90','red_per90':'reds_per_90','xG_for_per90':'team_xg_per_90',
                       'xG_against_per90':'team_xg_against_per_90','week':'gameweek','home':'is_at_home',
                       'position_y':'team_league_position'})
    
    return df[['full_name','team_name','player_position','current_fpl_cost','playing_time_min_percentage','xg_per_90','xag_per_90','blocks_per_90',
          'yellows_per_90','reds_per_90','team_xg_per_90','team_xg_against_per_90','gameweek','is_at_home','team_league_position']]

def get_players_with_points(gameweek=curr_gameweek-1):
    """
    Returns FPL players + their total points for a specific gameweek.
    """
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    response = requests.get(url)
    data = response.json()

    players = pd.DataFrame(data['elements'])
    teams = {team['id']: team['name'] for team in data['teams']}
    players['team_name'] = players['team'].map(teams)

    positions = {pos['id']: pos['singular_name'] for pos in data['element_types']}
    players['position'] = players['element_type'].map(positions)

    players_df = players[['id', 'first_name', 'second_name', 'team_name', 'position', 'now_cost']].copy()
    players_df['full_name'] = players_df['first_name'] + " " + players_df['second_name']

    points_list = []
    for pid in players_df['id']:
        p_url = f"https://fantasy.premierleague.com/api/element-summary/{pid}/"
        p_data = requests.get(p_url).json()

        history = p_data.get("history", [])
        gw_record = next((gw for gw in history if gw["round"] == gameweek), None)

        points_list.append(gw_record["total_points"] if gw_record else 0)

    players_df["gw_points"] = points_list
    
    return players_df[['full_name','gw_points']]

# --------------------------------------------------------------------------

warnings.simplefilter(action='ignore', category=FutureWarning)   

print("The current gameweek is: ",curr_gameweek)

gameweeks_seen = get_gameweeks_seen("/home/tars/Projects/fpl-oracle/data")

if curr_gameweek in gameweeks_seen:
    print("The gameweek has already been grabbed.")
else:
    # get the before gameweek data..
    df = join_it_all_together()
    df.to_csv(f'/home/tars/Projects/fpl-oracle/data/X_{curr_gameweek}.csv', index=False)

    # get the after last gameweek data...
    df_ = get_players_with_points()
    X = pd.read_csv(f"/home/tars/Projects/fpl-oracle/data/X_{curr_gameweek-1}.csv")
    filtered = df_[df_['full_name'].isin(X['full_name'])]
    filtered.to_csv(f'/home/tars/Projects/fpl-oracle/data/y_{curr_gameweek-1}.csv', index=False)

In [1]:
import sys
print(sys.executable)


/home/tars/Projects/fpl-oracle/venv311/bin/python


In [2]:
from understatapi import UnderstatClient

with UnderstatClient() as understat:
    league_player_data = understat.league(league="EPL").team(season="2023")

len(league_player_data), league_player_data[0]

(570,
 {'id': '8260',
  'player_name': 'Erling Haaland',
  'games': '31',
  'time': '2581',
  'goals': '27',
  'xG': '31.65399668365717',
  'assists': '5',
  'xA': '4.7517555598169565',
  'shots': '122',
  'key_passes': '29',
  'yellow_cards': '1',
  'red_cards': '0',
  'position': 'F S',
  'team_title': 'Manchester City',
  'npg': '20',
  'npxG': '25.564646281301975',
  'xGChain': '30.19725350290537',
  'xGBuildup': '3.128645434975624'})

In [11]:
from understatapi import UnderstatClient
import pandas as pd

with UnderstatClient() as understat:
    match_data = understat.league(league="EPL").get_match_data(season="2025")

df = pd.DataFrame(match_data)

"""
# compute xG totals
df["xg_home"] = df["xG"]["h"]
df["xg_away"] = df["xG"]["a"]

home_for = df.groupby("h_team")["xg_home"].sum()
away_for = df.groupby("a_team")["xg_away"].sum()

xg_for = home_for.add(away_for, fill_value=0)

home_against = df.groupby("h_team")["xg_away"].sum()
away_against = df.groupby("a_team")["xg_home"].sum()

xg_against = home_against.add(away_against, fill_value=0)

team_xg = pd.concat([xg_for, xg_against], axis=1).reset_index()
team_xg.columns = ["team", "xg_for", "xg_against"]
team_xg
"""
df.head(60)


,id,isResult,h,a,goals,xG,datetime,forecast
0,28778,True,"{'id': '87', 'title': 'Liverpool', 'short_titl...","{'id': '73', 'title': 'Bournemouth', 'short_ti...","{'h': '4', 'a': '2'}","{'h': '2.33007', 'a': '1.57303'}",2025-08-15 19:00:00,"{'w': '0.5498', 'd': '0.2276', 'l': '0.2226'}"
1,28779,True,"{'id': '71', 'title': 'Aston Villa', 'short_ti...","{'id': '86', 'title': 'Newcastle United', 'sho...","{'h': '0', 'a': '0'}","{'h': '0.318601', 'a': '1.40098'}",2025-08-16 11:30:00,"{'w': '0.0629', 'd': '0.2371', 'l': '0.7'}"
2,28780,True,"{'id': '220', 'title': 'Brighton', 'short_titl...","{'id': '228', 'title': 'Fulham', 'short_title'...","{'h': '1', 'a': '1'}","{'h': '1.44008', 'a': '0.902883'}",2025-08-16 14:00:00,"{'w': '0.5158', 'd': '0.3088', 'l': '0.1754'}"
3,28781,True,"{'id': '77', 'title': 'Sunderland', 'short_tit...","{'id': '81', 'title': 'West Ham', 'short_title...","{'h': '3', 'a': '0'}","{'h': '0.724368', 'a': '0.558948'}",2025-08-16 14:00:00,"{'w': '0.363', 'd': '0.391', 'l': '0.246'}"
4,28782,True,"{'id': '82', 'title': 'Tottenham', 'short_titl...","{'id': '92', 'title': 'Burnley', 'short_title'...","{'h': '3', 'a': '0'}","{'h': '1.97858', 'a': '0.734138'}",2025-08-16 14:00:00,"{'w': '0.7032', 'd': '0.1993', 'l': '0.0975'}"
5,28783,True,"{'id': '229', 'title': 'Wolverhampton Wanderer...","{'id': '88', 'title': 'Manchester City', 'shor...","{'h': '0', 'a': '4'}","{'h': '0.519436', 'a': '2.29903'}",2025-08-16 16:30:00,"{'w': '0.0404', 'd': '0.1179', 'l': '0.8417'}"
6,28784,True,"{'id': '80', 'title': 'Chelsea', 'short_title'...","{'id': '78', 'title': 'Crystal Palace', 'short...","{'h': '0', 'a': '0'}","{'h': '1.36793', 'a': '1.03414'}",2025-08-17 13:00:00,"{'w': '0.4404', 'd': '0.2929', 'l': '0.2667'}"
7,28785,True,"{'id': '249', 'title': 'Nottingham Forest', 's...","{'id': '244', 'title': 'Brentford', 'short_tit...","{'h': '3', 'a': '1'}","{'h': '1.7555', 'a': '1.32516'}",2025-08-17 13:00:00,"{'w': '0.4714', 'd': '0.2855', 'l': '0.2431'}"
8,28786,True,"{'id': '89', 'title': 'Manchester United', 'sh...","{'id': '83', 'title': 'Arsenal', 'short_title'...","{'h': '0', 'a': '1'}","{'h': '1.37653', 'a': '1.3304'}",2025-08-17 15:30:00,"{'w': '0.3408', 'd': '0.3074', 'l': '0.3518'}"
9,28787,True,"{'id': '245', 'title': 'Leeds', 'short_title':...","{'id': '72', 'title': 'Everton', 'short_title'...","{'h': '1', 'a': '0'}","{'h': '2.3812', 'a': '0.674338'}",2025-08-18 19:00:00,"{'w': '0.8116', 'd': '0.1394', 'l': '0.049'}"


# Phase 2: Model

- we will only start building this after 3 or 4 weeks of data collection
- Some things to keep in mind:
  - do we want to exclude from training all instances where a player scored 0 points? I feel this is a good way to get rid of instances where players just did not play. Even if a player plays and gets 0 points, this is a rare instance. Usually they will at least get 1.

In [143]:
import os
import re
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingRegressor

gameweek = 22

DATA_DIR = "/home/tars/Projects/fpl-oracle/data"
PRED_FILE = f"/var/www/reedrogers/data/predictions_2026_{gameweek}.csv"

def predict(gameweek: int):

    # -----------------------
    # Discover files
    # -----------------------
    files = os.listdir(DATA_DIR)
    pattern = re.compile(r'^(X|y)_(\d+)\.csv$')

    file_map = {}
    for f in files:
        match = pattern.match(f)
        if match:
            kind, num = match.groups()
            file_map.setdefault(int(num), {})[kind] = f

    # -----------------------
    # Build training data (ONLY completed weeks)
    # -----------------------
    merged_dfs = []

    for gw, pair in file_map.items():
        if gw < gameweek and "X" in pair and "y" in pair:
            X = pd.read_csv(os.path.join(DATA_DIR, pair["X"]))
            y = pd.read_csv(os.path.join(DATA_DIR, pair["y"]))

            print(
                gw,
                X.shape,
                X["full_name"].nunique()
                )


            merged = X.merge(y, on="full_name", how="inner")
            merged["gameweek"] = gw
            
            merged = merged[merged["playing_time_min_percentage"] > 80]
            
            merged_dfs.append(merged)
    if not merged_dfs:
        raise ValueError("No training data found.")

    train_df = pd.concat(merged_dfs, ignore_index=True)
    
    # -----------------------
    # Train model
    # -----------------------
    X_train = train_df.drop(columns=["gw_points", "full_name", "gameweek"])
    y_train = train_df["gw_points"]

    categorical = ["team_name", "player_position"]
    numeric = X_train.columns.difference(categorical)

    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
            ("num", "passthrough", numeric),
        ]
    )

    model = Pipeline(
        steps=[
            ("prep", preprocessor),
            ("gb", HistGradientBoostingRegressor(
                max_depth=6,
                learning_rate=0.05,
                max_iter=300,
                random_state=42
            ))
        ]
    )

    model.fit(X_train, y_train)

    # -----------------------
    # Predict on requested gameweek
    # -----------------------
    if gameweek not in file_map or "X" not in file_map[gameweek]:
        raise ValueError(f"X_{gameweek}.csv not found")

    X_latest = pd.read_csv(
        os.path.join(DATA_DIR, file_map[gameweek]["X"])
    )

    X_pred = X_latest.drop(columns=["full_name"])
    print(len(X_pred))
    preds = model.predict(X_pred)

    pred_df = pd.DataFrame({
        "full_name": X_latest["full_name"],
        "team_name": X_latest["team_name"],
        "position": X_latest["player_position"],
        "predicted_points": np.round(preds, 2),
        "actual_points": [""] * len(preds),
        "gameweek": gameweek
    })

    # -----------------------
    # Append predictions
    # -----------------------
    write_header = not os.path.exists(PRED_FILE)

    pred_df.to_csv(
        PRED_FILE,
        mode="a",
        index=False,
        header=write_header
    )

    print(f"Predictions for GW{gameweek} appended to predictions.csv")

In [144]:
predict(22)

13 (246, 15) 246
17 (246, 15) 246
18 (246, 15) 246
20 (246, 15) 246
21 (246, 15) 246
19 (246, 15) 246
16 (246, 15) 246
12 (246, 15) 246
246
Predictions for GW22 appended to predictions.csv


In [145]:
# join the actual points from the y files to the prediction files I created

PRED_DIR = "/var/www/reedrogers/data"
Y_DIR = "/home/tars/Projects/fpl-oracle/data"

pred_pattern = re.compile(r"predictions_2026_(\d+)\.csv")

for fname in os.listdir(PRED_DIR):
    match = pred_pattern.match(fname)
    if not match:
        continue

    gw = int(match.group(1))

    pred_path = os.path.join(PRED_DIR, fname)
    y_path = os.path.join(Y_DIR, f"y_{gw}.csv")

    if not os.path.exists(y_path):
        print(f"y_{gw}.csv not found — skipping")
        continue

    print(f"Processing GW{gw}")

    preds = pd.read_csv(pred_path)
    y = pd.read_csv(y_path)[["full_name", "gw_points"]]

    merged = preds.merge(
        y,
        on="full_name",
        how="left"
    )

    # Fill actual_points from gw_points
    merged["actual_points"] = merged["gw_points"]

    # Drop helper column
    merged = merged.drop(columns=["gw_points"])

    # Overwrite original prediction file
    merged.to_csv(pred_path, index=False)

    print(f"Updated {fname}")

Processing GW21
Updated predictions_2026_21.csv
Processing GW19
Updated predictions_2026_19.csv
Processing GW20
Updated predictions_2026_20.csv
y_22.csv not found — skipping


In [146]:
# now, script to join together all the predictions

DATA_DIR = "/var/www/reedrogers/data"
OUTPUT_FILE = os.path.join(DATA_DIR, "predictions.csv")

csv_files = [
    f for f in os.listdir(DATA_DIR)
    if f.endswith(".csv") and f != "predictions.csv"
]

dfs = []

for f in csv_files:
    path = os.path.join(DATA_DIR, f)
    df = pd.read_csv(path)
    df["source_file"] = f  # optional but VERY useful
    dfs.append(df)

if not dfs:
    raise ValueError("No CSV files found to merge.")

final_df = pd.concat(dfs, ignore_index=True)

final_df = final_df.drop('source_file', axis=1)

final_df.to_csv(OUTPUT_FILE, index=False)

print(f"Combined {len(csv_files)} files into {OUTPUT_FILE}")

Combined 4 files into /var/www/reedrogers/data/predictions.csv


In [147]:
# performance

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

PRED_FILE = "/var/www/reedrogers/data/predictions.csv"

# -----------------------
# Load + clean
# -----------------------
df = pd.read_csv(PRED_FILE)

# Ensure numeric
df["actual_points"] = pd.to_numeric(df["actual_points"], errors="coerce")
df["predicted_points"] = pd.to_numeric(df["predicted_points"], errors="coerce")

# Drop rows where actuals are missing
df = df.dropna(subset=["actual_points"])

# Sort by gameweek
df = df.sort_values("gameweek")

# -----------------------
# Compute cumulative metrics
# -----------------------
results = []

gameweeks = sorted(df["gameweek"].unique())

for gw in gameweeks:
    subset = df[df["gameweek"] <= gw]

    y_true = subset["actual_points"]
    y_pred = subset["predicted_points"]

    if len(y_true) < 2:
        continue  # R² undefined with <2 samples

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    results.append({
        "through_gameweek": gw,
        "n_samples": len(y_true),
        "MAE": round(mae, 3),
        "RMSE": round(rmse, 3),
        "R2": round(r2, 3)
    })

metrics_df = pd.DataFrame(results)

print(metrics_df)


   through_gameweek  n_samples    MAE   RMSE     R2
0                19        246  2.665  3.391 -0.467
1                20        492  2.441  3.227 -0.482
2                21        738  2.405  3.081 -0.816


In [ ]:
# What is next?

# - simple script to add in the points from last week (basically just join y_latestGameweek)

# - compute "live" model metrics.. MAE, RMSE, r-squared and Spearman rank correlation, with little blurbs for each. We want to monitor how this changes each week, 
#   so that means we need a second script that just will compute those metrics and save them before new data is added, and save that to metrics.csv

# - we want to add a little blurb for the metrics too, saying what they mean